In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
os.getcwd()

In [ ]:
os.chdir('drive/MyDrive/img')

In [ ]:
train_path = 'train'
valid_path = 'valid'
test_path = 'test'

In [ ]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(224,224), classes=['safe', 'unsafe'], batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=valid_path, target_size=(224,224), classes=['safe', 'unsafe'], batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=test_path, target_size=(224,224), classes=['safe', 'unsafe'], batch_size=10, shuffle=False)

In [ ]:
assert train_batches.n == 513
assert valid_batches.n == 14
assert test_batches.n == 102
assert train_batches.num_classes == valid_batches.num_classes == test_batches.num_classes ==2


In [ ]:
pip install --upgrade tensorflow keras numpy pandas sklearn pillow

In [ ]:
pip install -U scikit-learn scipy matplotlib

In [ ]:
imgs, labels = next(train_batches)

In [ ]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 10, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
plotImages(imgs)
print(labels)

In [ ]:
pip install boto3

In [ ]:
import cv2
import numpy as np
from io import BytesIO
from PIL import Image

In [ ]:
import boto3

In [ ]:
import boto3


def detect_hateful_images(imgs):
    # Create a Rekognition client

    client = boto3.client(
    'rekognition',
    region_name='us-east-1',
    aws_access_key_id='yourid',
    aws_secret_access_key='yourkey'
     )
    # Open image file
   #for imgs in imgs:
     # if imgs.dtype != np.uint8:
      #  imgs = (255 * imgs).astype(np.uint8)

    # Convert ndarray to PIL Image
    image = Image.fromarray(imgs)
    img_byte_arr = BytesIO()
    image.save(img_byte_arr, format='JPEG')  # You can change the format as needed
    img_byte_arr = img_byte_arr.getvalue()
    response = client.detect_moderation_labels(
            Image={'Bytes': img_byte_arr}
        )
    # Check for moderation labels
    if response['ModerationLabels']:
        for label in response['ModerationLabels']:
            print(f"Label: {label['Name']}, Confidence: {label['Confidence']}")

    return response['ModerationLabels']

# Usage

predicted_labels = detect_hateful_images(imgs)
if not labels:
    print("No inappropriate content detected.")
else:
    print("Inappropriate content detected.")


In [ ]:
predicted_labels = detect_hateful_images(imgs)
if not labels:
    print("No inappropriate content detected.")
else:
    print("Inappropriate content detected.")

In [ ]:
vgg16_model = tf.keras.applications.vgg16.VGG16()

In [ ]:
vgg16_model.summary()

In [ ]:
type(vgg16_model)

In [ ]:
Functional = Sequential()
for layer in vgg16_model.layers[:-1]:
    Functional.add(layer)

In [ ]:
from keras.models import Sequential

In [ ]:
Functional.summary()

In [ ]:
for layer in Functional.layers:
    layer.trainable = False

In [ ]:
Functional.add(Dense(units=2, activation='softmax'))

In [ ]:
Functional.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [ ]:
Functional.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy',])

In [ ]:
Functional.fit(x=train_batches, validation_data=valid_batches, epochs=5, verbose=2)

In [ ]:
predictions = Functional.predict(x=test_batches, verbose=0)

In [ ]:
test_batches.classes

In [ ]:
cm = confusion_matrix(y_true=test_batches.classes, y_pred=np.argmax(predictions, axis=-1))

In [ ]:
test_batches.class_indices

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
cm_plot_labels = ['safe', 'unsafe']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

In [ ]:
from sklearn.metrics import plot_confusion_matrix

In [ ]:
Functional.evaluate(x=test_batches)